In [1]:
# 10/22/24


In [2]:
# %pip install statsmodels

# %pip install ace_tools


In [3]:
import requests
import pandas as pd

import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from math import radians, cos, sin, asin, sqrt
import json
import os
from pandas import DataFrame, Series

import numpy as np

from sklearn.linear_model import LinearRegression

from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [4]:
# First, read all 4 of my csv files as dataframes.

part_level_demand = pd.read_csv('part_level_demand.csv')
distribution_centers = pd.read_csv('distribution_center_to_store_mapping.csv')
transportation_costs = pd.read_csv('transportation_cost_table.csv')
max_demands_per_dc = pd.read_csv('max_demands_per_dc_and_plant_supply.csv')


In [13]:
part_level_demand.head()


,product,store,date,demand
0,cordless screwdriver_1,Store_1,2022-10-24,88.0
1,cordless screwdriver_1,Store_1,2022-10-31,81.0
2,cordless screwdriver_1,Store_1,2022-11-07,57.0
3,cordless screwdriver_1,Store_1,2022-11-14,72.0
4,cordless screwdriver_1,Store_1,2022-11-21,95.0


In [17]:
distribution_centers.head()


,store,distribution_center
0,Store_28,Distribution_Center_3
1,Store_26,Distribution_Center_1
2,Store_27,Distribution_Center_2
3,Store_12,Distribution_Center_2
4,Store_22,Distribution_Center_2


In [15]:

# first I want to join part_level_demand with distribution_centers.
# I will use store as the key. I will use a left join to keep all the rows in part_level_demand.

# part_level_demand = pd.merge(part_level_demand, distribution_centers, on='store', how='left')

# part_level_demand


,product,store,date,demand,distribution_center
0,cordless screwdriver_1,Store_1,2022-10-24,88.0,Distribution_Center_1
1,cordless screwdriver_1,Store_1,2022-10-31,81.0,Distribution_Center_1
2,cordless screwdriver_1,Store_1,2022-11-07,57.0,Distribution_Center_1
3,cordless screwdriver_1,Store_1,2022-11-14,72.0,Distribution_Center_1
4,cordless screwdriver_1,Store_1,2022-11-21,95.0,Distribution_Center_1
...,...,...,...,...,...
94495,toolbox_3,Store_9,2024-09-23,26.0,Distribution_Center_4
94496,toolbox_3,Store_9,2024-09-30,51.0,Distribution_Center_4
94497,toolbox_3,Store_9,2024-10-07,73.0,Distribution_Center_4
94498,toolbox_3,Store_9,2024-10-14,50.0,Distribution_Center_4


In [7]:
transportation_costs.head()


,plant,product,Distribution_Center_1,Distribution_Center_2,Distribution_Center_3,Distribution_Center_4,Distribution_Center_5
0,plant_1,cordless screwdriver_1,1.623286,1.644768,1.276839,1.443802,1.106613
1,plant_2,cordless screwdriver_1,1.623286,1.644768,1.276839,1.443802,1.106613
2,plant_3,cordless screwdriver_1,1.467727,1.717629,1.504004,1.555256,1.369336
3,plant_1,cordless screwdriver_2,1.675242,1.264157,1.459132,1.268925,1.512000
4,plant_2,cordless screwdriver_2,1.724297,1.081003,1.854046,1.733505,1.422803


In [8]:
max_demands_per_dc.head()


,product,demand,plant_1,plant_2,plant_3
0,spirit level_2,3283.0,2299,1642,1314
1,toolbox_2,3916.0,2742,1958,1567
2,cordless screwdriver_1,3515.0,2461,1758,1406
3,screwdriver_2,3853.0,2698,1927,1542
4,spirit level_1,3467.0,2427,1734,1387


In [9]:
# let's see how many unique products there are

part_level_demand['product'].nunique()


30

In [54]:
# let's see how many unique stores there are

part_level_demand['store'].nunique()


30

In [36]:
distribution_centers.head()


,store,distribution_center
0,Store_28,Distribution_Center_3
1,Store_26,Distribution_Center_1
2,Store_27,Distribution_Center_2
3,Store_12,Distribution_Center_2
4,Store_22,Distribution_Center_2


In [37]:
transportation_costs.head()


,plant,product,Distribution_Center_1,Distribution_Center_2,Distribution_Center_3,Distribution_Center_4,Distribution_Center_5
0,plant_1,cordless screwdriver_1,1.623286,1.644768,1.276839,1.443802,1.106613
1,plant_2,cordless screwdriver_1,1.623286,1.644768,1.276839,1.443802,1.106613
2,plant_3,cordless screwdriver_1,1.467727,1.717629,1.504004,1.555256,1.369336
3,plant_1,cordless screwdriver_2,1.675242,1.264157,1.459132,1.268925,1.512000
4,plant_2,cordless screwdriver_2,1.724297,1.081003,1.854046,1.733505,1.422803


In [38]:
max_demands_per_dc.head()


,product,demand,plant_1,plant_2,plant_3
0,spirit level_2,3283.0,2299,1642,1314
1,toolbox_2,3916.0,2742,1958,1567
2,cordless screwdriver_1,3515.0,2461,1758,1406
3,screwdriver_2,3853.0,2698,1927,1542
4,spirit level_1,3467.0,2427,1734,1387


In [39]:
part_level_demand.dtypes


product     object
store       object
date        object
demand     float64
dtype: object

In [40]:

# Convert the 'date' column to a datetime object and sort the data
part_level_demand['date'] = pd.to_datetime(part_level_demand['date'])
part_level_demand = part_level_demand.sort_values(by=['product', 'store', 'date'])


In [41]:
part_level_demand.dtypes


product            object
store              object
date       datetime64[ns]
demand            float64
dtype: object


## One-step ahead forecast via Holt’s Winters Seasonal Method: Holt-Winters’ method is based on triple exponential smoothing and is able to account for both trend and seasonality.

In [42]:


# Function to forecast demand one week ahead using Holt-Winters method
def forecast_demand(data):
    # Prepare to store results
    forecast_results = []
    
    # Group by product and store to create separate time series
    grouped = data.groupby(['product', 'store'])
    
    # Iterate over each group
    for (product, store), group in grouped:
        # Sort by date and ensure we have a proper time series
        group = group.sort_values('date')
        
        # Fit the Holt-Winters model
        model = ExponentialSmoothing(group['demand'], 
                                     trend='add', 
                                     seasonal='add', 
                                     seasonal_periods=52,  # Assuming weekly seasonality over a year
                                     initialization_method='estimated').fit()
        
        # Forecast one step ahead (for the week of 10-28-2024)
        forecast = model.forecast(steps=1)
        
        # Store the forecast result
        forecast_results.append({
            'product': product,
            'store': store,
            'forecast_date': '2024-10-28',
            'forecasted_demand': forecast.iloc[0]
        })
    
    # Convert to DataFrame
    return pd.DataFrame(forecast_results)

# Run the forecast function
forecast_results = forecast_demand(part_level_demand)




/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base

In [43]:
product_forecast = forecast_results

product_forecast.head()

,product,store,forecast_date,forecasted_demand
0,cordless screwdriver_1,Store_1,2024-10-28,79.094837
1,cordless screwdriver_1,Store_10,2024-10-28,46.879475
2,cordless screwdriver_1,Store_11,2024-10-28,64.965423
3,cordless screwdriver_1,Store_12,2024-10-28,129.843638
4,cordless screwdriver_1,Store_13,2024-10-28,136.295957


Aggregate forecasts on distribution center level

In [23]:
distribution_centers.head()


,store,distribution_center
0,Store_28,Distribution_Center_3
1,Store_26,Distribution_Center_1
2,Store_27,Distribution_Center_2
3,Store_12,Distribution_Center_2
4,Store_22,Distribution_Center_2


In [27]:
distribution_centers.dtypes


store                  object
distribution_center    object
dtype: object

In [44]:
# Join forecast results with distribution centers to get the distribution center for each store.

# Merge the forecast results with the distribution centers
product_forecast = product_forecast.merge(distribution_centers, on='store', how='left') 

product_forecast.head()




,product,store,forecast_date,forecasted_demand,distribution_center
0,cordless screwdriver_1,Store_1,2024-10-28,79.094837,Distribution_Center_1
1,cordless screwdriver_1,Store_10,2024-10-28,46.879475,Distribution_Center_5
2,cordless screwdriver_1,Store_11,2024-10-28,64.965423,Distribution_Center_1
3,cordless screwdriver_1,Store_12,2024-10-28,129.843638,Distribution_Center_2
4,cordless screwdriver_1,Store_13,2024-10-28,136.295957,Distribution_Center_3


In [46]:
# Then aggregate the forecasted demand by distribution center for each product.

# Aggregate the forecasted demand by distribution center and product
product_forecast = product_forecast.groupby(['product', 'distribution_center']).sum().reset_index()

product_forecast.head()


/var/folders/bn/hy61x8_j5ng1fg7vpk14wkq00000gn/T/ipykernel_55512/1085051694.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  product_forecast = product_forecast.groupby(['product', 'distribution_center']).sum().reset_index()


,product,distribution_center,forecasted_demand
0,cordless screwdriver_1,Distribution_Center_1,537.894897
1,cordless screwdriver_1,Distribution_Center_2,871.932978
2,cordless screwdriver_1,Distribution_Center_3,403.402690
3,cordless screwdriver_1,Distribution_Center_4,498.683993
4,cordless screwdriver_1,Distribution_Center_5,490.670090


In [47]:
product_forecast



,product,distribution_center,forecasted_demand
0,cordless screwdriver_1,Distribution_Center_1,537.894897
1,cordless screwdriver_1,Distribution_Center_2,871.932978
2,cordless screwdriver_1,Distribution_Center_3,403.402690
3,cordless screwdriver_1,Distribution_Center_4,498.683993
4,cordless screwdriver_1,Distribution_Center_5,490.670090
...,...,...,...
145,toolbox_3,Distribution_Center_1,653.538048
146,toolbox_3,Distribution_Center_2,536.121683
147,toolbox_3,Distribution_Center_3,572.476512
148,toolbox_3,Distribution_Center_4,641.508982
